In [ ]:
import shutil
import os
import os.path as op
from collections import OrderedDict

from eptools.people import (get_profiles_registry,
                            fetch_profiles_files,
                            contact_from_dict)

from eptools.badges.factory import (BadgeFactory, 
                                    get_badge_role,
                                    prepare_badge_pdf,
                                    get_badge_template_file)

In [ ]:
output_dir = op.expanduser('~/Desktop/eps_badges')
os.makedirs(output_dir, exist_ok=True)

CURRENT_EUROPYTHON_FIRST_TICKET_ID = 3732

factory = BadgeFactory(output_dir)

In [ ]:
fetch_data = True

# fetch the data from the DB
if fetch_data:
    profiles_file, talks_file = fetch_profiles_files()
    print(profiles_file, talks_file)

In [ ]:
def create_badge(contact, roles, factory):

    # create the SVG file
    svg_file = factory.generate_badge_svg(contact, roles)

    # convert the SVG to PDF
    pdf_file = prepare_badge_pdf(svg_file, doublepages=False)

    # make both badge faces and save the result in its place
    outdir = op.join(factory.out_dir, get_badge_role(roles).name)
    os.makedirs(outdir, exist_ok=True)
    
    return outdir, pdf_file


import hashlib

def file_cmp(file1, file2, hash_func=hashlib.sha224):
    os.utime(file1)
    os.utime(file2) # touch
    
    hash1 = hash_func(open(file1, 'rb').read()).hexdigest()
    hash2 = hash_func(open(file2, 'rb').read()).hexdigest()
    return hash1 == hash2

In [ ]:
# load and prepared the profiles
profiles = get_profiles_registry()

In [ ]:
new_badges = []
for profile in profiles:
   
    if profile.pop('frozen'):
        continue

    if profile['id'] < CURRENT_EUROPYTHON_FIRST_TICKET_ID:
        continue
    
    contact = contact_from_dict(profile)

    roles   = list(profiles.get_roles_of(contact.email, 
                                         name=contact.name, 
                                         surname=contact.surname))
       
    outdir, pdf_file = create_badge(contact, roles, factory)
    
    result_file = op.join(outdir, op.basename(pdf_file))

    move = False
    #if op.exists(result_file):
    #    if not file_cmp(pdf_file, result_file): # not equal
    #        move = True
    #else:
    #    move = True
    if not op.exists(result_file):
        move = True

    if move:
        _ = shutil.move(pdf_file, result_file)
        new_badges.append(result_file)
        print(' ✓')
    else:
        print(' X')

In [ ]:
import subprocess
from glob import glob
from itertools import zip_longest

def grouper(iterable, n, fillvalue=None):
    "Collect data into fixed-length chunks or blocks"
    # grouper('ABCDEFG', 3, 'x') --> ABC DEF Gxx"
    args = [iter(iterable)] * n
    return zip_longest(*args, fillvalue=fillvalue)

In [ ]:
len(list(grouper(badge_files, 4)))

In [ ]:
badge_files = glob(op.join(output_dir, 'out', '**', '*.pdf'))

cmd = 'pdfjam {input} --nup 2x2 --papersize "{{23cm,33cm}}" --noautoscale true -o {output}'

output_template = op.join(output_dir, 'to_print', 'profiles', 'ep2017_badges_{}.pdf')

for idx, badges in enumerate(grouper(badge_files, 1, fillvalue='')):
    
    output = output_template.format(idx)
    
    make_badge = cmd.format(input=' '.join(badges), output=output)
    
    print(make_badge)
    subprocess.call(make_badge, shell=True)

In [ ]:
badge_files = glob(op.join(output_dir, 'out', '**', '*.pdf'))

cmd = 'pdfjam {input} --nup 1x1 --doublepages true --papersize "{{23cm,33cm}}" --noautoscale true -o {output}'

output_template = op.join(output_dir, 'to_print_1x1_big', 'profiles', '{}')

for idx, badge in enumerate(badge_files): 
    
    output = output_template.format(op.basename(badge))
    
    make_badge = cmd.format(input=badge, output=output)
    
    print(make_badge)
    subprocess.call(make_badge, shell=True)

In [ ]:
badge_files = glob('/home/alexandre/Projects/ep-tools/eptools/badges/data/ep2017/blanks/*.pdf')

cmd = 'pdfjam {input} --nup 1x1 --doublepages true --papersize "{{23cm,33cm}}" --noautoscale true -o {output}'

output_template = op.join(output_dir, 'to_print_1x1_big', 'blanks', '{}')

for idx, badge in enumerate(badge_files): 
    
    output = output_template.format(op.basename(badge))
    
    make_badge = cmd.format(input=badge, output=output)
    
    print(make_badge)
    subprocess.call(make_badge, shell=True)